In [1]:
# Define package
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory
import math

In [2]:
# Create INPUT_POINTS data
input_points=pd.DataFrame(columns=['Point_ID','X_Coord','Y_Coord','Tumor_Flg'])
input_points['Point_ID']=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
input_points['X_Coord']=[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]
input_points['Y_Coord']=[1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]
input_points['Tumor_Flg']=[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]

In [24]:
m = AbstractModel()

m.SEEDS = Set(initialize = range(1, 4))
m.CELLS = Set(initialize = range(1, 17))

m.XSEED = Var(m.SEEDS, bounds = (0, 4), domain = NonNegativeReals)
m.YSEED = Var(m.SEEDS, bounds = (0, 4), domain = NonNegativeReals)

m.XCELL = Param(m.CELLS)
m.YCELL = Param(m.CELLS)
m.TFLAG = Param(m.CELLS)

def distance(m, c, s):
    return ((m.XSEED[s] - m.XCELL[c])**2  + (m.YSEED[s] - m.YCELL[c])**2)**0.5

m.DISTANCE = Expression(m.CELLS, m.SEEDS, initialize = distance)

def exposure(m, c, s):
    return 1 / (m.DISTANCE[c, s] + 0.01)

m.EXPOSURE = Expression(m.CELLS, m.SEEDS, initialize = exposure)

m.minExp = 3
def min_exposure_constraint(m, c):
    if not m.TFLAG[c]:
        return Constraint.Skip
    return quicksum(m.EXPOSURE[c, s] for s in m.SEEDS) >= m.minExp

m.minExposureConstraint = Constraint(m.CELLS, rule = min_exposure_constraint)

def total_healthy_exposure(m):
    return quicksum(m.EXPOSURE[c,s] * (1 - m.TFLAG[c]) for c in m.CELLS for s in m.SEEDS)

m.totalHealthyExposure = Objective(rule = total_healthy_exposure, sense = minimize)

In [25]:
instanceData = {None: {
    'XCELL' : input_points.set_index(['Point_ID']).to_dict()['X_Coord'],
    'YCELL' : input_points.set_index(['Point_ID']).to_dict()['Y_Coord'],
    'TFLAG' : input_points.set_index(['Point_ID']).to_dict()['Tumor_Flg']
}}
print(instanceData)
instance = m.create_instance(instanceData)

{None: {'XCELL': {1: 1, 2: 1, 3: 1, 4: 1, 5: 2, 6: 2, 7: 2, 8: 2, 9: 3, 10: 3, 11: 3, 12: 3, 13: 4, 14: 4, 15: 4, 16: 4}, 'YCELL': {1: 1, 2: 2, 3: 3, 4: 4, 5: 1, 6: 2, 7: 3, 8: 4, 9: 1, 10: 2, 11: 3, 12: 4, 13: 1, 14: 2, 15: 3, 16: 4}, 'TFLAG': {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 1, 14: 0, 15: 0, 16: 0}}}


In [26]:
opt = SolverFactory('ipopt')
results = opt.solve(instance)

In [27]:
instance.display()

Model unknown

  Variables:
    XSEED : Size=3, Index=SEEDS
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          1 :     0 :  4.000000038158469 :     4 : False : False : NonNegativeReals
          2 :     0 : 1.4102193180075375 :     4 : False : False : NonNegativeReals
          3 :     0 : 2.5382688635638675 :     4 : False : False : NonNegativeReals
    YSEED : Size=3, Index=SEEDS
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          1 :     0 : 0.5615295368839917 :     4 : False : False : NonNegativeReals
          2 :     0 : 2.0428463314179064 :     4 : False : False : NonNegativeReals
          3 :     0 : 2.9155848364250394 :     4 : False : False : NonNegativeReals

  Objectives:
    totalHealthyExposure : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 17.93939409647452

  Constraints:
    minExposureConstraint : Size=5
        Key : Lower : Body               : Upper
          